In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=5

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=5


In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
OUT_PATH: str = '../results/manifolds'

In [3]:
DATA_PATH: str = '../data/imdb'
DATASETS: list = [
    ('train', 'sample.train'),
    ('test', 'sample.test')
]

In [4]:
DIMS: list = [768, 576, 384, 192, 96, 48, 24, 12, 6, 3]
MODELS: list = [
    ('base', 'bert-base-uncased'),
    ('textattack', 'textattack/bert-base-uncased-imdb'),
    ('fabriceyhc', 'fabriceyhc/bert-base-uncased-imdb'),
    ('wakaka', 'Wakaka/bert-finetuned-imdb')
]

In [5]:
### Load Datasets into memory

In [6]:
data_config: dict = {
    'polarities': {
      "negative": 0,
      "positive": 1
    },
    'data_label': 'text',
    'target_label': 'sentiment'
}

In [1]:
from typing import Dict
from modules import Data

datasets: Dict[str, Data] = {
    label: Data(data_path=f"{DATA_PATH}.{name}.csv", **data_config)
    for label, name in DATASETS
}

ModuleNotFoundError: No module named 'classifier'

In [8]:
for label, dataset in datasets.items():
    display(dataset.data)
    display(dataset.data['sentiment'].value_counts(normalize=True))

,text,sentiment
0,The acting in this movie was superb. As an ama...,positive
1,Watching That Lady In Ermine I was wondering w...,negative
2,"Why?!! This was an insipid, uninspired and emb...",negative
3,"Although the plot was a bit sappy at times, an...",positive
4,This is one of the few comedies I can watch ag...,positive
...,...,...
1240,This film has a special place in my heart as t...,negative
1241,OK me and a friend rented this a few days ago ...,negative
1242,Five-year-old Michael sees his mother getting ...,negative
1243,Every movie critic and metal head hated this m...,positive


positive    0.515663
negative    0.484337
Name: sentiment, dtype: float64

,text,sentiment
0,one of my favorite lines in Shakespeare.i.e. *...,positive
1,A poor basketball movie. A gruff coach with a ...,negative
2,"Based on the book ""Space Vampires"" by Colin Wi...",positive
3,There are about ten minutes about half way thr...,negative
4,>>> Great News there is a BBC DVD release sche...,positive
...,...,...
1235,Sheba Shayne (Pam Grier) receives a telegram i...,negative
1236,"On one level, Hari Om is a film using a famili...",positive
1237,"OK, so I gotta start this review by saying i w...",negative
1238,My daughter already wrote a review of this mov...,positive


negative    0.514516
positive    0.485484
Name: sentiment, dtype: float64

In [9]:
### Load Encoder into Memory

In [10]:
encoder_config: dict = {
    'layers': [11]
}

In [11]:
from classifier._neural import Encoder

encoders: Dict[str, Encoder] = {
    label: Encoder({**{'model': ref}, **encoder_config})
    for label, ref in MODELS
}

In [12]:
### Compute manifolds and measure centroid distance and cluster dispersion

In [13]:
import numpy as np
import pandas as pd

In [14]:
from sklearn.manifold import MDS

def manifold_reduction(data: np.ndarray, dim: int = 3) -> np.ndarray:
    return MDS(n_components=dim).fit_transform(data)

In [15]:
from scipy.spatial import distance

def metric_computation(record: dict, groups: pd.core.groupby.GroupBy) -> None:

    # compute centroid means and dispersion for each cluster
    for label, group in groups:
            record[f'centroid_{label}'] = np.mean(np.stack(group['reduced_embeds'].tolist(), axis=0), axis=0).tolist()
            record[f'dispersion_{label}'] = np.sum(distance.pdist(group['reduced_embeds'].tolist()))

    record['distance'] = distance.cdist([record['centroid_positive']], [record['centroid_negative']]).item(0)

In [16]:
from typing import Generator
import torch

def reduce_analyse(data: pd.DataFrame, col: str, dim: list, default_dim: int = 768) -> Generator:

    embed_col: np.ndarray = torch.stack(data[col].tolist()).numpy()

    for d in dim:

        # create record to keep row data
        record: dict = {'dim': d}

        # if reduction size is equal to encoder output dim, skip manifold reduction
        if d == default_dim:
            data['reduced_embeds'] = list(embed_col)
        else:
            data['reduced_embeds'] = list(manifold_reduction(embed_col, dim=d))

        metric_computation(record, data.groupby(dataset.target_label))

        yield record

In [17]:
output_cols: list = ['dim', 'dispersion_positive', 'dispersion_negative', 'distance']

In [18]:
results: Dict[str, pd.DataFrame] = {}

In [19]:
for enc_label, encoder in encoders.items():
    for data_label, dataset in datasets.items():
        encoder.df_encode(dataset.data, col=dataset.data_label)
        results[f'{data_label}.{enc_label}'] = pd.DataFrame.from_records(
            list(reduce_analyse(
                dataset.data, encoder.col_name, DIMS,
                default_dim=encoder.dim)
            )
        )

In [20]:
for label, dataset in results.items():
    results[label][output_cols].to_csv(f'{OUT_PATH}/analysis.{label}.csv')
    display(results[label][output_cols])

,dim,dispersion_positive,dispersion_negative,distance
0,768,1.780549e+06,1.511064e+06,1.215327
1,576,1.781745e+06,1.512164e+06,1.145085
2,384,1.781667e+06,1.512090e+06,1.141992
3,192,1.781040e+06,1.511510e+06,1.160019
4,96,1.779856e+06,1.510311e+06,1.189642
5,48,1.777581e+06,1.508217e+06,1.226943
6,24,1.771273e+06,1.502219e+06,1.374433
7,12,1.759527e+06,1.490951e+06,1.538300
8,6,1.736990e+06,1.467700e+06,1.600850
9,3,1.685935e+06,1.413828e+06,1.318991


,dim,dispersion_positive,dispersion_negative,distance
0,768,1.555449e+06,1.680930e+06,1.173917
1,576,1.556606e+06,1.682037e+06,1.100364
2,384,1.556409e+06,1.681852e+06,1.107525
3,192,1.556013e+06,1.681282e+06,1.115471
4,96,1.554800e+06,1.680074e+06,1.145750
5,48,1.552124e+06,1.677225e+06,1.220297
6,24,1.546479e+06,1.670641e+06,1.363173
7,12,1.535531e+06,1.657568e+06,1.551356
8,6,1.508335e+06,1.625345e+06,1.909222
9,3,1.471065e+06,1.563120e+06,1.532055


,dim,dispersion_positive,dispersion_negative,distance
0,768,2.080085e+06,1.761406e+06,7.908210
1,576,2.080430e+06,1.761678e+06,7.910161
2,384,2.080154e+06,1.761415e+06,7.911783
3,192,2.079200e+06,1.760514e+06,7.918288
4,96,2.077196e+06,1.758636e+06,7.930053
5,48,2.072722e+06,1.754415e+06,7.959053
6,24,2.063231e+06,1.745557e+06,8.016581
7,12,2.041085e+06,1.724909e+06,8.148785
8,6,1.985406e+06,1.673575e+06,8.459311
9,3,1.818896e+06,1.530549e+06,9.251512


,dim,dispersion_positive,dispersion_negative,distance
0,768,1.854011e+06,1.996711e+06,7.360264
1,576,1.854369e+06,1.996996e+06,7.361293
2,384,1.854081e+06,1.996713e+06,7.363674
3,192,1.853283e+06,1.995866e+06,7.368602
4,96,1.851391e+06,1.993884e+06,7.382469
5,48,1.847360e+06,1.989571e+06,7.410671
6,24,1.838861e+06,1.980605e+06,7.467608
7,12,1.819118e+06,1.959699e+06,7.600142
8,6,1.769348e+06,1.906282e+06,7.921906
9,3,1.620701e+06,1.757998e+06,8.749805


,dim,dispersion_positive,dispersion_negative,distance
0,768,1.803098e+06,1.919905e+06,22.903667
1,576,1.803555e+06,1.920438e+06,22.903796
2,384,1.803466e+06,1.920355e+06,22.903874
3,192,1.803199e+06,1.920147e+06,22.904074
4,96,1.802619e+06,1.919707e+06,22.904353
5,48,1.801146e+06,1.918544e+06,22.905467
6,24,1.798087e+06,1.916102e+06,22.908108
7,12,1.789945e+06,1.909628e+06,22.915375
8,6,1.766668e+06,1.889920e+06,22.937814
9,3,1.663101e+06,1.804950e+06,23.122424


,dim,dispersion_positive,dispersion_negative,distance
0,768,1.892527e+06,2.477473e+06,20.314917
1,576,1.893023e+06,2.477948e+06,20.315408
2,384,1.892957e+06,2.477871e+06,20.315493
3,192,1.892713e+06,2.477653e+06,20.315670
4,96,1.892192e+06,2.477131e+06,20.316334
5,48,1.890933e+06,2.476012e+06,20.317581
6,24,1.888090e+06,2.473019e+06,20.320730
7,12,1.880559e+06,2.465967e+06,20.329727
8,6,1.858032e+06,2.443480e+06,20.361308
9,3,1.762150e+06,2.352780e+06,20.590927


,dim,dispersion_positive,dispersion_negative,distance
0,768,2.433107e+06,1.920625e+06,11.704244
1,576,2.433626e+06,1.920946e+06,11.707232
2,384,2.433385e+06,1.920715e+06,11.707926
3,192,2.432659e+06,1.919993e+06,11.710755
4,96,2.430984e+06,1.918393e+06,11.717164
5,48,2.427405e+06,1.914943e+06,11.730650
6,24,2.419387e+06,1.907246e+06,11.762039
7,12,2.399986e+06,1.888622e+06,11.842136
8,6,2.344958e+06,1.839915e+06,12.067655
9,3,2.200707e+06,1.708544e+06,12.634450


,dim,dispersion_positive,dispersion_negative,distance
0,768,2.101408e+06,2.188378e+06,11.394895
1,576,2.101885e+06,2.188790e+06,11.397200
2,384,2.101656e+06,2.188540e+06,11.398204
3,192,2.100948e+06,2.187746e+06,11.401071
4,96,2.099524e+06,2.186181e+06,11.405367
5,48,2.096108e+06,2.182420e+06,11.420528
6,24,2.088705e+06,2.174193e+06,11.452305
7,12,2.070243e+06,2.154695e+06,11.532118
8,6,2.023704e+06,2.104365e+06,11.734304
9,3,1.893067e+06,1.967083e+06,12.306471
